# 키워드 연관 분석 진행
1. 관광지 전체 키워드 리스트를 생성
2. 키워드 리스트를 SBERT를 이용해 임베딩
3. 관광지 별 블로그글을 SBERT를 이용해 임베딩
4. 관광지 별로 전체 키워드들의 연관도 Cosain 유사도를 이용해 추출

In [22]:
import pandas as pd
import numpy as np
import re

from collections import Counter
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

okt = Okt()

# 한국어를 포함하고 있는 다국어 SBERT load
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [4]:
df = pd.read_parquet('append_keyword.parquet')
df

,source,content,postdate,platform,keyword
0,아르떼뮤지엄,강릉 여행 드라이브 코스 강원도 바다 강릉 아르떼뮤지엄 동생 함께 강원도 드라이브 ...,20220225,naver,"강릉시,즐거움,주차장,평창동계올림픽,자연,중앙시장,경험,청소년,음식,유명해서,디지털..."
1,아르떼뮤지엄,#아르떼뮤지엄 #아르떼뮤지엄강릉 아르떼뮤지엄 강릉 다시 어가 강릉 여행기 제주 시작...,20220217,naver,"주차장,관람객,함정,아르떼뮤지엄강릉,많은데,미디어,접근성,도시,정원,전시관,타이밍,..."
2,아르떼뮤지엄,안녕하세요 헤라 입니다 드디어 제주여행 이야기 끝나고 다녀온 가족여행 포스팅 있게 ...,20220228,naver,"입장권,가족여행,바쁘,낭만,여친,정원,인기,블로그,보름달,노란색,체험,금요일,부지런..."
3,아르떼뮤지엄,지난주 가족 제주여행 해안도로 산책 제주도 다녀왔어요 와중 제주 아르떼뮤지엄 맛집 ...,20220127,naver,"대만족,식당,멸치볶음,공간,제주공항,우정,중독,산책,메뉴판,영업,부모님,은빛,주차장..."
4,아르떼뮤지엄,안녕하세요 😄#국내여행 #아기랑여행 #아기랑국내여행목포에서 아침 먹이 숙소 오금 뒹...,20220312,naver,"여수꽃돌게장1번가게장맛집이,음식,공룡,박람회,전기차,주차장,아기랑국내여행목포에서,볶..."
...,...,...,...,...,...
78926,김택화미술관,인정 많고 좋은 신흥리 김택화미술관 개관 했다 한라산 남긴 김택화 화백 작품 바탕 ...,20200805,brunch,"김택화,서울,비평가,학교,헌신,한국,신흥리,가구,스케치,귀하,쓰레기,공간,사이클링,..."
78927,김택화미술관,어느덧 여행 마지막 되었다 조식 먹고 숙소 맡기고 나오는데 벌써 아쉬운 마음이 든다...,20210325,brunch,"김택화,항쟁,서점,의존,인터뷰,테마파크,스케치,인터넷,버스,목재,거리,공간,역사,황..."
78928,김택화미술관,오늘 아가 생일 돌잔치 치렀고 남편 서울 있으니 잔치 분위기 없지만 하루 종일 뭉클...,20211124,brunch,"요구르트,고생,서울,존경,생일,수프,계란,소주,열정,없었던,고향,청귤,문화,샌드위치..."
78929,김택화미술관,세번 베케 트래블 테크 분야 뉴스 리핑 입니다 트립소 제주도 어디 가봤니 체결 계약...,20211119,brunch,"론칭김택화,경쟁,가상세계,관광,다채롭,시장,투어,단독,콘텐츠,전략,창업,고객,백신,..."


In [5]:
source = df['source'].unique()

# 관광지 별 불용어 제거 한 워드 카운팅

In [6]:
key_dict={}
for s in source:
    key_list = []
    remove_set = {'네이버','제주도','블로그','포스트','제주여행',
                  '제주공항','제주도여행','대한민국','한국','제주','제주시'}
    
    for key in df[df['source']==s]['keyword']:
        key_list.extend(key.split(','))
    
    
    key_list = [k for k in key_list if k not in remove_set]    
    
    count = Counter(key_list)
    
    key_count = count.most_common(100)
    
    key_dict[s]= [w for w,n in key_count[:100]]
    #print(s)

In [246]:
li=['에코랜드 테마파크',
'성산일출봉(UNESCO 세계자연유산)',
'천아숲길 천아계곡',
'종달리수국길',
'녹산로유채꽃길']

In [7]:
key_dict

{'아르떼뮤지엄': ['공간',
  '미디어아트',
  '여행',
  '미디어',
  '강릉',
  '전시관',
  '강릉시',
  '토끼',
  '경험',
  '입장권',
  '거울',
  '음악',
  '박람회',
  '트릭',
  '색칠',
  '영상',
  '정글',
  '계획',
  '주차장',
  '정원',
  '기도',
  '체험',
  '자연',
  '동물',
  '사진',
  '코끼리',
  '녹색',
  '전시',
  '국내',
  '인기',
  '겨울',
  '디지털',
  '거리',
  '디자인',
  '데이트',
  '피아노',
  '메뉴',
  '기념품',
  '서울',
  '확장',
  '도시',
  '바람',
  '판매',
  '기린',
  '어린이',
  '가족',
  '서양미술',
  '검색',
  '관광',
  '향기',
  '동백꽃',
  '세계',
  '가격',
  '시간',
  '사진찍기',
  '호랑이',
  '관광지',
  '사슴',
  '연출',
  '시공간',
  '안녕하세요',
  '마스크',
  '전라남도',
  '번개',
  '박물관',
  '기술',
  '공장',
  '인터넷',
  '동영상',
  '전시장',
  '관람객',
  '택시',
  '인스타',
  '색감',
  '온라인',
  '아르떼뮤지엄강릉',
  '전시회',
  '음식',
  '스캔',
  '성인',
  '영업',
  '아동',
  '일요일',
  '강릉아르떼뮤지엄',
  '꽃들이',
  '새로운',
  '꽃밭',
  '청소년',
  '지도',
  '유럽',
  '구매',
  '미술관',
  '평의',
  '아트',
  '광고',
  '산책',
  '감탄',
  '영원한',
  '식당',
  '운영'],
 '산방산': ['메뉴',
  '유채꽃',
  '여행',
  '공간',
  '용머리해안',
  '주차장',
  '바람',
  '사진',
  '거리',
  '식당',
  '용머리',
  '음식',
  '계획',
  '판매',
  '기도',


In [319]:
candidate=[]
for i in li:
    candidate.extend(key_dict[i])

# 중복 제거
temp_set = set(candidate)
# list로 재변환
candidate =list(temp_set)

In [260]:
def preprocessing(content):
    try:
        result=''
        tokenized_doc = okt.pos(content)
        global stop_word
        #print(tokenized_doc)
        tokenized = ' '.join([w for w,t in tokenized_doc if  t not in ['Verb'] and w not in stop_word])
        
    except Exception as e:
        print(e)
        print(content)
    return tokenized
        

In [302]:
def association(source):
    print(source)
    s_re = re.sub('[^-가-힣0-9]',' ',source)
    global stop_word
    stop_word = ['있는','있고','있으니','입니다.','이렇게','있을','제주특별자치도',source,s_re.replace(' ','')]
    stop_word.extend(s_re.split(' '))
    
    doc_list = list(df[df['source']==source]['content'].map(preprocessing))
    
    
    
    doc_embedding = model.encode(doc_list)
    candidate_embeddings = model.encode(candidate)
    print('글의 갯수:',len(doc_list))
    
    distances = cosine_similarity(doc_embedding,candidate_embeddings)
    
    # 행방향 평균
    dis_mean = distances.mean(axis=0)
    #print(len(dis_mean))
    print(dis_mean.argsort()[-50:])
    return [(candidate[index],dis_mean[index]) for index in dis_mean.argsort()[-50:]]

In [303]:
association(li[0])

에코랜드 테마파크
글의 갯수: 423
[267 123 273  14  12 183 163 151  79  59  65 228 230 168 165  99 144  81
  21 187   3  52 279 174 133 103 251  35  19  60 227 193 219   5  46 152
 149 274 221  40 292  36 241 252 203 141   8  38 129 283]


[('바람', 0.29702252),
 ('단풍', 0.29955834),
 ('향기', 0.30023658),
 ('벚꽃축제', 0.30073217),
 ('기차', 0.30170473),
 ('종달리전망대', 0.30238473),
 ('메밀꽃', 0.3033157),
 ('꽃길', 0.30361223),
 ('홍보', 0.30371207),
 ('구경', 0.3038986),
 ('한라산', 0.30511308),
 ('아름다운', 0.3077672),
 ('단풍놀이', 0.30883452),
 ('서귀포시', 0.3105855),
 ('안녕하세요', 0.3113098),
 ('열차', 0.3145622),
 ('입구', 0.31477),
 ('동백꽃', 0.31658083),
 ('월정리', 0.31692505),
 ('일본군', 0.32169715),
 ('공항', 0.32398972),
 ('성산일출봉', 0.32682523),
 ('한라산국립공원', 0.32905203),
 ('구매', 0.3315372),
 ('김녕', 0.33270144),
 ('아름다운길', 0.33272418),
 ('드라이브', 0.33423263),
 ('벚꽃', 0.33494598),
 ('디즈니랜드', 0.33598635),
 ('해녀', 0.34133455),
 ('매력', 0.34659514),
 ('영업', 0.347249),
 ('당근', 0.34914973),
 ('한라산둘레길', 0.35295865),
 ('천아숲길', 0.35975048),
 ('사려니숲길', 0.36040795),
 ('투어', 0.37207663),
 ('성산일출축제', 0.38027802),
 ('입장권', 0.39348993),
 ('제주세계자연유산센터', 0.3991514),
 ('서울', 0.39936346),
 ('버스정류장', 0.39955035),
 ('조천읍', 0.4114237),
 ('녹산로유채꽃도로', 0.4162572),
 ('제주유채꽃축제', 0.42012608

In [268]:
association(li[1])

성산일출봉(UNESCO 세계자연유산)
글의 갯수: 438


[('해안도로', 0.32755962),
 ('바람', 0.32816747),
 ('도로', 0.32990208),
 ('홍보', 0.33031768),
 ('꽃길', 0.33064786),
 ('아름다운', 0.33120087),
 ('디즈니랜드', 0.33258778),
 ('풍차', 0.33328518),
 ('둘레길', 0.33432212),
 ('안녕하세요', 0.3343793),
 ('단풍놀이', 0.3360097),
 ('볼드윈', 0.3363423),
 ('구경', 0.33804977),
 ('벚꽃', 0.3388391),
 ('아름다운길', 0.3402092),
 ('단풍', 0.34351274),
 ('종달리전망대', 0.344953),
 ('입구', 0.34580734),
 ('동백꽃', 0.34757894),
 ('월정리', 0.34878692),
 ('일본군', 0.3515592),
 ('매력', 0.35322288),
 ('공항', 0.35419968),
 ('한라산', 0.35493264),
 ('서귀포시', 0.3549721),
 ('한라산국립공원', 0.35887897),
 ('김녕', 0.35948363),
 ('해녀', 0.36059543),
 ('구매', 0.36241612),
 ('드라이브', 0.36642885),
 ('영업', 0.3691998),
 ('당근', 0.37007478),
 ('성산일출봉', 0.3722879),
 ('사려니숲길', 0.384773),
 ('한라산둘레길', 0.39304167),
 ('천아숲길', 0.39469877),
 ('투어', 0.40310076),
 ('입장권', 0.4072963),
 ('성산일출축제', 0.4131713),
 ('서울', 0.42013752),
 ('버스정류장', 0.43143696),
 ('제주유채꽃축제', 0.43532187),
 ('녹산로유채꽃도로', 0.43950358),
 ('제주세계자연유산센터', 0.44209757),
 ('조천읍', 0.4527166

In [269]:
association(li[2])

천아숲길 천아계곡
글의 갯수: 458


[('해안도로', 0.29456204),
 ('족욕탕', 0.2949083),
 ('도로', 0.29852736),
 ('구경', 0.2985278),
 ('아름다운', 0.29904947),
 ('일본군', 0.2995405),
 ('바람', 0.29983842),
 ('종달고망난돌쉼터', 0.29989925),
 ('입구', 0.30095485),
 ('둘레길', 0.30141616),
 ('한라산', 0.30172133),
 ('메밀꽃', 0.30218196),
 ('벚꽃축제', 0.30333027),
 ('종달리전망대', 0.30375868),
 ('김녕', 0.30559453),
 ('단풍놀이', 0.30866593),
 ('단풍', 0.30890995),
 ('꽃길', 0.30897093),
 ('종달리해안도로', 0.31138432),
 ('동백꽃', 0.3143927),
 ('서귀포시', 0.31465152),
 ('공항', 0.31549096),
 ('월정리', 0.31714734),
 ('한라산국립공원', 0.32117105),
 ('구매', 0.3255672),
 ('드라이브', 0.32936192),
 ('벚꽃', 0.33041295),
 ('매력', 0.33352724),
 ('영업', 0.33589664),
 ('해녀', 0.33620206),
 ('성산일출봉', 0.34023502),
 ('아름다운길', 0.34099185),
 ('당근', 0.34730712),
 ('한라산둘레길', 0.35075122),
 ('투어', 0.36278883),
 ('서울', 0.36870992),
 ('입장권', 0.37258148),
 ('천아숲길', 0.38193196),
 ('성산일출축제', 0.3827394),
 ('사려니숲길', 0.3875337),
 ('제주세계자연유산센터', 0.38952306),
 ('버스정류장', 0.39504457),
 ('조천읍', 0.39635658),
 ('제주유채꽃축제', 0.42487064),
 ('녹산로유

In [270]:
association(li[3])

종달리수국길
글의 갯수: 531


[('유채꽃길', 0.27195048),
 ('유채', 0.2743132),
 ('종달리해안도로', 0.2752686),
 ('입구', 0.27809164),
 ('홍보', 0.27828935),
 ('디즈니랜드', 0.2796811),
 ('유채꽃', 0.28003687),
 ('서귀포시', 0.28078225),
 ('구경', 0.28190038),
 ('언니', 0.2839745),
 ('아름다', 0.28486288),
 ('공항', 0.2859673),
 ('안녕하세요', 0.2905675),
 ('성산일출봉', 0.29133376),
 ('벚꽃축제', 0.29191417),
 ('꽃길', 0.29237473),
 ('한라산국립공원', 0.2989026),
 ('향기', 0.29906255),
 ('월정리', 0.2994298),
 ('족욕탕', 0.29962254),
 ('드라이브', 0.3007126),
 ('일본군', 0.3007142),
 ('아름다운', 0.30117497),
 ('메밀꽃', 0.30549148),
 ('구매', 0.30615667),
 ('동백꽃', 0.30750835),
 ('김녕', 0.31090102),
 ('한라산둘레길', 0.3128201),
 ('영업', 0.31972852),
 ('천아숲길', 0.3236965),
 ('아름다운길', 0.3260342),
 ('벚꽃', 0.32826102),
 ('사려니숲길', 0.3304252),
 ('투어', 0.33457264),
 ('당근', 0.337582),
 ('해녀', 0.3456647),
 ('매력', 0.34963986),
 ('성산일출축제', 0.35408762),
 ('입장권', 0.35626587),
 ('제주세계자연유산센터', 0.36288655),
 ('버스정류장', 0.3667963),
 ('서울', 0.37401795),
 ('조천읍', 0.37636974),
 ('녹산로유채꽃도로', 0.39269644),
 ('제주유채꽃축제', 0.41014063

In [271]:
association(li[4])

녹산로유채꽃길
글의 갯수: 477


[('종달리전망대', 0.26492646),
 ('공항', 0.27257517),
 ('아름다', 0.27299765),
 ('서귀포시', 0.27314958),
 ('종달리해안도로', 0.27336347),
 ('일본군', 0.2738973),
 ('구경', 0.2771643),
 ('아름다운', 0.27765024),
 ('김녕', 0.27822372),
 ('드라이브', 0.2789085),
 ('디즈니랜드', 0.28003863),
 ('유채', 0.28301564),
 ('단풍놀이', 0.28320342),
 ('족욕탕', 0.28425303),
 ('홍보', 0.28786704),
 ('한라산둘레길', 0.2879684),
 ('향기', 0.2894846),
 ('한라산국립공원', 0.28973663),
 ('구매', 0.29190308),
 ('꽃들이', 0.30119047),
 ('유채꽃', 0.3052089),
 ('월정리', 0.30598792),
 ('유채꽃길', 0.3080555),
 ('성산일출봉', 0.30807626),
 ('영업', 0.31103685),
 ('해녀', 0.31350532),
 ('아름다운길', 0.31718877),
 ('꽃길', 0.3173474),
 ('메밀꽃', 0.31817728),
 ('동백꽃', 0.32236427),
 ('매력', 0.32356083),
 ('투어', 0.3251095),
 ('당근', 0.3275687),
 ('천아숲길', 0.32897213),
 ('사려니숲길', 0.33611527),
 ('버스정류장', 0.3389433),
 ('벚꽃축제', 0.34239134),
 ('입장권', 0.34341565),
 ('조천읍', 0.3461262),
 ('벚꽃', 0.3464052),
 ('제주세계자연유산센터', 0.35701364),
 ('서울', 0.3637249),
 ('성산일출축제', 0.37745315),
 ('성산읍', 0.40299347),
 ('녹산로유채꽃도로', 0.4066

In [305]:

tour = li[0]
tour_re = re.sub('[^-가-힣0-9]',' ',tour)
stop_word = ['있는','있고','있으니','입니다.','이렇게','있을','제주특별자치도',tour,tour_re.replace(' ','')]
stop_word.extend(tour_re.split(' '))

doc_list = list(df[df['source']==tour]['content'].map(preprocessing))



doc_embedding = model.encode(doc_list)
candidate_embeddings = model.encode(candidate)
print('글의 갯수:',len(doc_list))

distances = cosine_similarity(doc_embedding,candidate_embeddings)

# 행방향 평균
dis_mean = distances.mean(axis=0)
#print(len(dis_mean))
print(dis_mean.argsort()[-50:])
[(candidate[index],dis_mean[index]) for index in dis_mean.argsort()[-50:]]

글의 갯수: 423
[267 123 273  14  12 183 163 151  79  59  65 228 230 168 165  99 144  81
  21 187   3  52 279 174 133 103 251  35  19  60 227 193 219   5  46 152
 149 274 221  40 292  36 241 252 203 141   8  38 129 283]


[('바람', 0.29702252),
 ('단풍', 0.29955834),
 ('향기', 0.30023658),
 ('벚꽃축제', 0.30073217),
 ('기차', 0.30170473),
 ('종달리전망대', 0.30238473),
 ('메밀꽃', 0.3033157),
 ('꽃길', 0.30361223),
 ('홍보', 0.30371207),
 ('구경', 0.3038986),
 ('한라산', 0.30511308),
 ('아름다운', 0.3077672),
 ('단풍놀이', 0.30883452),
 ('서귀포시', 0.3105855),
 ('안녕하세요', 0.3113098),
 ('열차', 0.3145622),
 ('입구', 0.31477),
 ('동백꽃', 0.31658083),
 ('월정리', 0.31692505),
 ('일본군', 0.32169715),
 ('공항', 0.32398972),
 ('성산일출봉', 0.32682523),
 ('한라산국립공원', 0.32905203),
 ('구매', 0.3315372),
 ('김녕', 0.33270144),
 ('아름다운길', 0.33272418),
 ('드라이브', 0.33423263),
 ('벚꽃', 0.33494598),
 ('디즈니랜드', 0.33598635),
 ('해녀', 0.34133455),
 ('매력', 0.34659514),
 ('영업', 0.347249),
 ('당근', 0.34914973),
 ('한라산둘레길', 0.35295865),
 ('천아숲길', 0.35975048),
 ('사려니숲길', 0.36040795),
 ('투어', 0.37207663),
 ('성산일출축제', 0.38027802),
 ('입장권', 0.39348993),
 ('제주세계자연유산센터', 0.3991514),
 ('서울', 0.39936346),
 ('버스정류장', 0.39955035),
 ('조천읍', 0.4114237),
 ('녹산로유채꽃도로', 0.4162572),
 ('제주유채꽃축제', 0.42012608

In [353]:
doc_embedding2 = model.encode([doc_list[0]])
candidate_embeddings2 = model.encode(['성산일출축제','김녕굴','인스타','제주유채꽃축제','버스정류장','김통정'])
candidate2=['김통정','김녕굴','인스타','제주유채꽃축제','버스정류장','성산일출축제']
distances2 = cosine_similarity(doc_embedding2,candidate_embeddings2)

In [354]:
distances2

array([[0.34068203, 0.6497669 , 0.5991863 , 0.4209882 , 0.44660547,
        0.6937244 ]], dtype=float32)

In [357]:
len(candidate_embeddings2[0])

768

# word2vec 연관 분석
1. 각 관광지의 문서 불용어 전처리 진행
2. 각 문서에 존재하는 단어들의 벡터값의 평균을 구하여 해당 문서의 벡터값을 연산
 (단어 벡터를 모두 더한 값을 문서 길이로 나눔)
3. 원하는 키워드를 입력으로 넣으면 키워드로 이루어진 하나의 문장을 만듦
4. 해당 문장과 유사한 문서 벡터값을 가진 관광지를 추천해줌.
5. 참고(https://wikidocs.net/102705)

In [12]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
def tokenizedOkt(sentences):
    okt = Okt()
    total_tokenized_list = []
    # 해당 관광지에 관한 content들 토큰화
    tqdmDf = tqdm(sentences)
    count = 0
    
    for tourInfo in tqdmDf:
        tqdmDf.set_description(f'Processing tokenized')
        stop_word = ['있는','있고','있으니','입니다.','이렇게','있을', df['source'][count], df['source'][count].replace(' ',''),'네이버','제주도','블로그','포스트','제주여행',
                  '제주공항','제주도여행','대한민국','한국','제주','제주시']
        tokenized_doc = okt.pos(tourInfo)
        tokenized_list = [w for w,t in tokenized_doc if t not in ['Verb'] and w not in stop_word]
        total_tokenized_list.append(tokenized_list)
        count+=1
        #if count % 10000 == 0:
            #pd.DataFrame(data=total_tokenized_list).to_csv('tok.csv')
            
        
    return total_tokenized_list

In [4]:
#tok_sentence = tokenizedOkt(df['content'])

import json
with open('top100_tok_dic.json', 'r',encoding='UTF-8') as f:

    json_data = json.load(f)

tok_sentence =json_data['sentence']


NameError: name 'tokjson_data' is not defined

In [5]:
tok_sentence =json_data['sentence']

In [7]:
word2vec_model = KeyedVectors.load_word2vec_format("jeju_word2Vec_model")

In [50]:
def keyword_recommend(keywords):
    result_df.loc[len(test_df)] = ['input', keywords]
    # 각 문서들의 벡터 리스트
    document_embedding_list = []
    # 각 문서에 대해서
    for line in result_df['keyword']:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]
        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)
    # 코사인 유사도 매트릭스 생성
    cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
    books = result_df['source']
    # 해당 인풋의 인덱스 호출
    indices = pd.Series(result_df.index, index = result_df['source']).drop_duplicates()
    idx = indices['input']
    # 입력된 키워드와 유사한 관광지 1개 추천
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:10]
    book_indices = [i[0] for i in sim_scores]
    # 전체 데이터 프레임에서 해당 행 추출
    recommend = books.iloc[book_indices].reset_index()
    return recommend

In [57]:
keyword_recommend('바다 서쪽 해녀')

C:\Users\82104\AppData\Local\Temp/ipykernel_13160/3172149673.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in word2vec_model.wv.vocab:


,index,source
0,68,이호테우말등대
1,38,신창풍차해안도로
2,55,서우봉
3,53,노을해안로
4,52,금능해수욕장
5,31,광치기해변
6,88,애월한담해안산책로
7,51,닭머르해안길
8,36,비양도


In [58]:
result_df

,source,keyword
0,아르떼뮤지엄,미디어아트 미디어 전시관 강릉시 토끼 경험 입장권 거울 음악 박람회 트릭 색칠 영상...
1,산방산,유채꽃 용머리해안 바람 사진 거리 식당 용머리 음식 계획 기도 영업 자연 가격 디저...
2,함덕해수욕장,해수욕장 식당 바람 산책 해변 음식 거리 사진 여름 가족 고기 기도 영업 맛집 흑돼...
3,사계해변,산방산 사진 바람 해변 용머리해안 자연 인스타 식당 거리 시간 계획 송악산 디저트 ...
4,섭지코지,성산읍 성산일출봉 바람 유채꽃 거리 산책 식당 자연 음식 동쪽 사진 성리 드라마 겨...
...,...,...
96,녹산로유채꽃길,벚꽃 유채꽃길 축제 유채꽃 조랑말체험공원 노란 프라자 사진 거리 계획 바람 관광객 ...
97,스누피가든,정원 기념품 만화 캐릭터 찰리 산책 사진 거리 겨울 자연 계획 관광지 여름 체험 야...
98,제주센트럴파크,센트럴파크 한세상 거리 계획 인천 세대 인천광역시 산책 시장 사업 경쟁률 디저트 분...
99,김택화미술관,김택화 신흥로 커피 미술관 동영상 신흥리 사진 전시 자연 조천읍 인스타그램 소주 디...


In [55]:
result_df =pd.read_parquet('top100_keyword.parquet')
result_df

,source,keyword
0,아르떼뮤지엄,미디어아트 미디어 전시관 강릉시 토끼 경험 입장권 거울 음악 박람회 트릭 색칠 영상...
1,산방산,유채꽃 용머리해안 바람 사진 거리 식당 용머리 음식 계획 기도 영업 자연 가격 디저...
2,함덕해수욕장,해수욕장 식당 바람 산책 해변 음식 거리 사진 여름 가족 고기 기도 영업 맛집 흑돼...
3,사계해변,산방산 사진 바람 해변 용머리해안 자연 인스타 식당 거리 시간 계획 송악산 디저트 ...
4,섭지코지,성산읍 성산일출봉 바람 유채꽃 거리 산책 식당 자연 음식 동쪽 사진 성리 드라마 겨...
...,...,...
95,종달리수국길,사진 종달리해안도로 성산일출봉 동쪽 산책 계획 바람 바다 기도 식당 여름 자연 거리...
96,녹산로유채꽃길,벚꽃 유채꽃길 축제 유채꽃 조랑말체험공원 노란 프라자 사진 거리 계획 바람 관광객 ...
97,스누피가든,정원 기념품 만화 캐릭터 찰리 산책 사진 거리 겨울 자연 계획 관광지 여름 체험 야...
98,제주센트럴파크,센트럴파크 한세상 거리 계획 인천 세대 인천광역시 산책 시장 사업 경쟁률 디저트 분...


In [ ]:
result_df

In [18]:
test_df= result_df.copy()

In [390]:
def preprocessing(text):
    result = ' '.join(text)
    return result

In [ ]:
test_df['keyword']=test_df.keyword.map(preprocessing)

In [391]:
test_df

,source,keyword
0,아르떼뮤지엄,공간 미디어아트 여행 미디어 강릉 전시관 강릉시 토끼 경험 입장권 거울 음악 박람회...
1,산방산,메뉴 유채꽃 여행 공간 용머리해안 주차장 바람 사진 거리 식당 용머리 음식 계획 판...
2,함덕해수욕장,여행 메뉴 해수욕장 식당 공간 바람 산책 주차장 해변 음식 거리 사진 여름 가족 고...
3,사계해변,산방산 여행 사진 바람 메뉴 해변 공간 주차장 용머리해안 자연 인스타 안녕하세요 검...
4,섭지코지,성산읍 성산일출봉 여행 바람 주차장 메뉴 유채꽃 공간 거리 산책 식당 자연 음식 동...
...,...,...
95,종달리수국길,여행 사진 종달리해안도로 성산일출봉 동쪽 공간 산책 계획 검색 바람 바다 기도 주차...
96,녹산로유채꽃길,벚꽃 유채꽃길 축제 유채꽃 여행 조랑말체험공원 노란 프라자 사진 거리 계획 바람 관...
97,스누피가든,공간 정원 기념품 만화 여행 주차장 캐릭터 찰리 산책 사진 거리 겨울 자연 계획 판...
98,제주센트럴파크,센트럴파크 서울 공간 메뉴 안녕하세요 한세상 거리 계획 인천 세대 인천광역시 여행 ...


In [392]:
result_df = test_df.copy()

In [ ]:
result_df['keyword'][0]